In [ ]:
# Dogs vs Cats Classification using Two Models (Simple CNN and MobileNetV2 (Transfer Learning))

In [ ]:
#Importing the libraries


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Loads images,Resize's them,normalise pixels
import os, pathlib   # Navigate Downloaded folders

print("TensorFlow_Version : ", tf.__version__)



In [ ]:
# Downloading the Cats & Dogs Dataset

import tensorflow_datasets as tfds

dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
train_ds = dataset['train']


In [ ]:
# Only use a subset for faster training

LIMIT_TRAIN = 4000     # Number of training samples
LIMIT_VAL = 1000       # Number of validation samples

train_data_raw = train_ds.take(LIMIT_TRAIN)
val_data_raw = train_ds.skip(LIMIT_TRAIN).take(LIMIT_VAL)


In [ ]:
# PreProcessing (Resize + Normalize)

IMG_SIZE = 160

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_data = train_data_raw.map(preprocess).batch(32).prefetch(1)  # MAP apply a function to every item in the dataset
val_data = val_data_raw.map(preprocess).batch(32).prefetch(1)


In [ ]:
#Building Model 1 – Simple CNN

model_1 = tf.keras.models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(1, activation='sigmoid')
])


In [ ]:
# configure  the model for training

model_1.compile(
    optimizer="Adam",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_1.summary()


In [ ]:
history_1 = model_1.fit(
    train_data,
    epochs=5,
    validation_data=val_data
)


In [ ]:
# Visualize Training – Accuracy & Loss Curves

import matplotlib.pyplot as plt

# 1. Extract values from history object
acc = history_1.history['accuracy']
val_acc = history_1.history['val_accuracy']
loss = history_1.history['loss']
val_loss = history_1.history['val_loss']

epochs = range(1, len(acc) + 1)

# 2. Create figure
plt.figure(figsize=(12, 5))

# 3. Accuracy graph
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training vs Validation Accuracy (Model 1 - Simple CNN)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 4. Loss graph
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training vs Validation Loss (Model 1 - Simple CNN)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
#Evaluate CNN on Validation Data

val_loss, val_acc = model_1.evaluate(val_data)
print("Final Validation Loss (Model 1):", val_loss)
print("Final Validation Accuracy (Model 1):", val_acc)


In [ ]:
# MODEL 2 - MobileNetV2

IMG_SIZE = 160

def preprocess_mnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

train_data_m2 = train_data_raw.map(preprocess_mnet).batch(32).prefetch(1)
val_data_m2   = val_data_raw.map(preprocess_mnet).batch(32).prefetch(1)


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # freeze base model weights

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs)

model_2.summary()


In [ ]:
model_2.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
#Train (Transfer Learning)

history_2 = model_2.fit(
    train_data_m2,
    epochs=5,
    validation_data=val_data_m2
)


In [ ]:
# Plotting Accuracy & Loss

import matplotlib.pyplot as plt

acc2 = history_2.history['accuracy']
val_acc2 = history_2.history['val_accuracy']
loss2 = history_2.history['loss']
val_loss2 = history_2.history['val_loss']

epochs2 = range(1, len(acc2) + 1)

plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs2, acc2, label='Training Accuracy (Model 2)')
plt.plot(epochs2, val_acc2, label='Validation Accuracy (Model 2)')
plt.title('Model 2 - Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs2, loss2, label='Training Loss (Model 2)')
plt.plot(epochs2, val_loss2, label='Validation Loss (Model 2)')
plt.title('Model 2 - Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
#Evaluate on Validation Data
val_loss_2, val_acc_2 = model_2.evaluate(val_data_m2)
print("Final Validation Loss (Model 2):", val_loss_2)
print("Final Validation Accuracy (Model 2):", val_acc_2)


In [ ]:
#Comparing Model 1 vs Model 2

m1_train_last = history_1.history['accuracy'][-1]
m1_val_last   = history_1.history['val_accuracy'][-1]

m2_train_last = history_2.history['accuracy'][-1]
m2_val_last   = history_2.history['val_accuracy'][-1]

print("Model 1 - Train:", m1_train_last, " Val:", m1_val_last)
print("Model 2 - Train:", m2_train_last, " Val:", m2_val_last)


In [ ]:
#Real Image Prediction (Dog vs Cat)

from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

uploaded = files.upload()

for file_name in uploaded.keys():
    # Load and display the image
    image = Image.open(file_name)
    plt.imshow(image)
    plt.axis('off')

    # Preprocess image
    image = image.resize((160, 160))
    img_array = np.array(image)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # make it batch of 1

    # Predict
    prediction = model_2.predict(img_array)[0][0]

    if prediction > 0.5:
        print(f"Prediction: Dog ({prediction:.2f})")
    else:
        print(f"Prediction: Cat ({1 - prediction:.2f})")
